In [266]:
%%time

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re

CPU times: total: 0 ns
Wall time: 11.5 ms


In [382]:
spells_df = pd.read_csv("dnd_spells_update.csv")

spells_df

,Name,Level,School,Casting Time,Duration,Range,Area,Attack,Save,Damage/Effect,Ritual,Concentration,Verbal,Somatic,Material,Components,Source,Description
0,Acid Splash,0,Conjuration,1 Action,Instantaneous,60 ft,NaN,NaN,DEX,Acid,NaN,NaN,Y,Y,NaN,NaN,Players Handbook,You hurl a bubble of acid. Choose one or two c...
1,Blade Ward,0,Abjuration,1 Action,1 Round,Self,NaN,NaN,NaN,Combat (...),NaN,NaN,Y,Y,NaN,NaN,Players Handbook,You extend your hand and trace a sigil of ward...
2,Booming Blade,0,Evocation,1 Action,1 Round,Self (5 ft),NaN,Melee,NaN,Thunder (...),NaN,NaN,NaN,Y,Y,a melee weapon worth at least 1 sp,Tasha's Cauldron of Everything,You brandish the weapon used in the spell’s ca...
3,Chill Touch,0,Necromancy,1 Action,1 Round,120 ft,NaN,Ranged,NaN,Necrotic,NaN,NaN,Y,Y,NaN,NaN,Players Handbook,"You create a ghostly, skeletal hand in the spa..."
4,Control Flames,0,Transmutation,1 Action,Instantaneous,60 ft,cube 5 ft,NaN,NaN,Control,NaN,NaN,NaN,Y,NaN,NaN,Elemental Evil,You choose nonmagical flame that you can see w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,Time Stop,9,Transmutation,1 Action,Instantaneous,Self,NaN,NaN,NaN,Control,NaN,NaN,Y,NaN,NaN,NaN,Players Handbook,You briefly stop the flow of time for everyone...
516,True Polymorph,9,Transmutation,1 Action,1 Hour,30 ft,NaN,NaN,WIS,Buff (...),NaN,Y,Y,Y,Y,"a drop of mercury, a dollop of gum arabic, and...",Players Handbook,Choose one creature or nonmagical object that ...
517,True Resurrection,9,Necromancy,1 Hour,Instantaneous,Touch,NaN,NaN,NaN,Healing,NaN,NaN,Y,Y,Y,a sprinkle of holy water and diamonds worth at...,Players Handbook,You touch a creature that has been dead for no...
518,Weird,9,Illusion,1 Action,1 Minute,120 ft,sphere 30 ft,NaN,WIS,Psychic,NaN,Y,Y,Y,NaN,NaN,Players Handbook,Drawing on the deepest fears of a group of cre...


In [400]:
# clean the data / format it better

# start by replacing NaN with False for some columns
spells_df.loc[:,['Verbal','Somatic','Material','Concentration', 'Ritual']] = spells_df.loc[:,['Verbal','Somatic','Material','Concentration','Ritual']].fillna(False)

# replace the other Nan with blanks
spells_df = spells_df.fillna("")

# change Y to true
spells_df.loc[spells_df['Verbal'] == 'Y','Verbal'] = True
spells_df.loc[spells_df['Somatic'] == 'Y','Somatic'] = True
spells_df.loc[spells_df['Material'] == 'Y','Material'] = True
spells_df.loc[spells_df['Concentration'] == 'Y','Concentration'] = True
spells_df.loc[spells_df['Ritual'] == 'Y','Ritual'] = True

# make sure level is read as integer
spells_df["Level"] = spells_df["Level"].astype(int)

# remove parantheses from beginning and end of components (if necessary)
spells_df['Components'].str.strip('()')

# dictionary of shorthand for the sources
sources_shorthand = {'Players Handbook':'PHB',
           'Monster Manual': 'MM',
           'Elemental Evil': 'EE',
           'Xanathars Guide To Everything':'XGtE',
           "Tasha's Cauldron of Everything": 'TCoE',
           "Lost Laboratory of Kwalish": 'LLoK',
           "Mordenkainen's Tome of Foes": 'MToF',
           "Acquisitions Incorporated": 'AI',
           'Matt': 'Matt',
           'Lindsey':'Lindsey',
           'Matt & Lindsey': 'ML',
           'Other Homebrew': 'Other HB',
           "Unearthed Arcana": 'UI',
           "Basic Rules": 'Basic',
           "Explorer's Guide to Wildemount":"EGtW",
            "Guildmasters' Guide to Ravnica":'GGtR'}

# replace full source name with shorthand name
spells_df.replace({"Source": sources_shorthand},inplace=True)
           


spells_df

,Name,Level,School,Casting Time,Duration,Range,Area,Attack,Save,Damage/Effect,Ritual,Concentration,Verbal,Somatic,Material,Components,Source,Description
0,Acid Splash,0,Conjuration,1 Action,Instantaneous,60 ft,,,DEX,Acid,False,False,True,True,False,,PHB,You hurl a bubble of acid. Choose one or two c...
1,Blade Ward,0,Abjuration,1 Action,1 Round,Self,,,,Combat (...),False,False,True,True,False,,PHB,You extend your hand and trace a sigil of ward...
2,Booming Blade,0,Evocation,1 Action,1 Round,Self (5 ft),,Melee,,Thunder (...),False,False,False,True,True,a melee weapon worth at least 1 sp,TCoE,You brandish the weapon used in the spell’s ca...
3,Chill Touch,0,Necromancy,1 Action,1 Round,120 ft,,Ranged,,Necrotic,False,False,True,True,False,,PHB,"You create a ghostly, skeletal hand in the spa..."
4,Control Flames,0,Transmutation,1 Action,Instantaneous,60 ft,cube 5 ft,,,Control,False,False,False,True,False,,EE,You choose nonmagical flame that you can see w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,Time Stop,9,Transmutation,1 Action,Instantaneous,Self,,,,Control,False,False,True,False,False,,PHB,You briefly stop the flow of time for everyone...
516,True Polymorph,9,Transmutation,1 Action,1 Hour,30 ft,,,WIS,Buff (...),False,True,True,True,True,"a drop of mercury, a dollop of gum arabic, and...",PHB,Choose one creature or nonmagical object that ...
517,True Resurrection,9,Necromancy,1 Hour,Instantaneous,Touch,,,,Healing,False,False,True,True,True,a sprinkle of holy water and diamonds worth at...,PHB,You touch a creature that has been dead for no...
518,Weird,9,Illusion,1 Action,1 Minute,120 ft,sphere 30 ft,,WIS,Psychic,False,True,True,True,False,,PHB,Drawing on the deepest fears of a group of cre...


In [402]:
%%time

class Spells_Known:
    
    
    def __init__(self, current_char_level: int, spells_df: pd.DataFrame):
        self.current_char_level = current_char_level # type: int
        
        if spells_df is None:
            spells_df = pd.read_csv("dnd_spell_info.csv")
        self.spells_df = spells_df # type: pd.DataFrame
        
        
        self.spell_list = [[],[],[],[],[],[],[],[],[],[]] # type: list of lists

    
    def add_spell(self, spell_name: str):
        
        # all lowercase so case-insensitive
        spell_name = spell_name.lower()
        
        try:
            # get index of the spell in the spells df
            spell_index = spells_df.index[self.spells_df['Name'].apply(str.lower) == spell_name].astype('int')
            
            # use index to get spell level
            spell_level = int(self.spells_df["Level"].values[spell_index])
            
            # add to the spell list!
            self.spell_list[spell_level].append(spell_name)
            
        # in case user enters invalid spell name, makes a typing error, spell not in the database, etc.
        except TypeError:
            print(f"Spell name '{spell_name}' not valid. Please try another.")
            
            return None
        

         
        return self.spell_list
    
    def compile_spells(self):
        
        # get column names 
        my_cols = self.spells_df.columns.tolist()
        
        # initialize df
        my_spellbook = pd.DataFrame(columns = my_cols)
        
        # loop through spells matrix
        for i in self.spell_list:
            
            # alphabetize through each spell level
            i = sorted(i)
            
            for spell in i:
                
                # get index of the spell in the spells df
                spell_index = spells_df.index[self.spells_df['Name'].apply(str.lower) == spell].tolist()
                
                # get all data for this spell
                entry = self.spells_df.iloc[spell_index,:]
                
                # add it to the spellbook
                my_spellbook = my_spellbook.append([entry])
        
        
        # fix indices
        my_spellbook.index = range(0, len(my_spellbook))
        
        #
        my_spellbook["Level"] = my_spellbook["Level"].replace(0, 'cantrip')
        
        return my_spellbook
    
    def _for_borders(self):
        
        # initialize list for indices
        my_indices = []
        
        # initialize spell counter
        counter = 0
        
        # go through the spells list
        for i in self.spell_list:
            
            # alphabetize through each spell level
            i = sorted(i)
            
            if len(i) == 0:
                break
            
            else:
                
                last_spell = len(i)

                # add to indices
                my_indices.append(last_spell + counter)

                for spell in i:
                    counter += 1
            
            
        return my_indices
        
        
        
        

CPU times: total: 0 ns
Wall time: 5.29 ms


In [403]:
# make a class for Piper's spells
x = Spells_Known(4, spells_df)

# add all of Piper's spells (Can be in any order)
x.add_spell('healing word')
x.add_spell('Cure Wounds')
x.add_spell("Light")
x.add_spell('suggestion')
x.add_spell('Zone of truth')
x.add_spell('augury')
x.add_spell('message')
x.add_spell('sanctuary')
x.add_spell('Detect evil and good')
x.add_spell('Detect magic')
x.add_spell('Identify')
x.add_spell('Illusory script')
x.add_spell('Command')
x.add_spell('inflict wounds')
x.add_spell('Sacred flame')
x.add_spell('guidance')
x.add_spell('spare the dying')
x.add_spell('ritual of the bones of black and gold')

[['light', 'message', 'sacred flame', 'guidance', 'spare the dying'],
 ['healing word',
  'cure wounds',
  'sanctuary',
  'detect evil and good',
  'detect magic',
  'identify',
  'illusory script',
  'command',
  'inflict wounds'],
 ['suggestion',
  'zone of truth',
  'augury',
  'ritual of the bones of black and gold'],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [404]:
spellbook = x.compile_spells()


spellbook.head()

,Name,Level,School,Casting Time,Duration,Range,Area,Attack,Save,Damage/Effect,Ritual,Concentration,Verbal,Somatic,Material,Components,Source,Description
0,Guidance,cantrip,Divination,1 Action,1 Minute,Touch,,,,Buff,False,True,True,True,False,,PHB,You touch one willing creature. Once before th...
1,Light,cantrip,Evocation,1 Action,1 Hour,Touch,sphere 20 ft,,DEX,Creation (...),False,False,True,False,True,a firefly or phosphorescent moss),PHB,You touch one object that is no larger than 10...
2,Message,cantrip,Transmutation,1 Action,1 Round,120 ft,,,,Communication (...),False,False,True,True,True,a short piece of copper wire),PHB,You point your finger toward a creature within...
3,Sacred Flame,cantrip,Evocation,1 Action,Instantaneous,60 ft,,,DEX,Radiant,False,False,True,True,False,,PHB,Flame-like radiance descends on a creature tha...
4,Spare the Dying,cantrip,Necromancy,1 Action,Instantaneous,Touch,,,,Healing,False,False,True,True,False,,PHB,You touch a living creature that has 0 hit poi...


In [405]:
def style_negative(v, props=''):
    return props if v == False else None


def highlight_true(s, props=''):
    return np.where(s == True, props, '')


# dictionary for color-coded schools of magic
color_matcher = {'Evocation': '#E83B43', 'Abjuration': '#2E4FC9', 'Necromancy': '#000000', 'Conjuration': '#E2B118', 'Divination': '#AE1AEA',
                 'Illusion': '#1AEAB4', 'Enchantment': '#F83DDC', 'Transmutation': '#F5B068'}


def color_cells(x):
    return 'color: ' + x.map(
        # Associate Values to a given colour code
        color_matcher
    ).fillna('black')  # Fill unmapped values with default


# let it be formatted and hopefully pretty
pretty_spells = spellbook.style.set_caption("Piper's Spells").set_table_styles([{"selector":"tbody tr:nth-child(even)","props":[("background-color","#E5E6E7")]}])\
    .set_properties(**{'text-align': 'left','border': '1.3px solid black'}).apply(color_cells).apply(highlight_true, props='font-weight:bold').applymap(style_negative, props='color:black;')\
              .applymap(lambda v: 'opacity: 20%;' if (v == False) else None).set_properties(subset=['Description'], **{'width': '300px'})
    
    
# write to html
pretty_spells.to_html('piper_spells.html')


In [407]:
%%time

# write to excel file, can do even prettier formatting there (or google sheets)
pretty_spells.to_excel('pipers_spells2.xlsx', engine='openpyxl')

CPU times: total: 93.8 ms
Wall time: 500 ms
